# Robust Clustering Evaluation Pipeline

### 🧩 Problem Statement
- **What**: We need to segment 5000 B2B SaaS accounts into meaningful groups based on usage behavior (e.g., logins, feature adoption).
- **Why**: To enable targeted marketing (churn prevention, upselling) without having pre-existing labels (no ground truth).
- **Relevance**: In the real world, data is often unlabeled ("Unsupervised Learning"). We need a robust way to find patterns that aren't just random noise.

### 🪜 Steps to Solve the Problem
1.  **Synthetic Data Generation**: Create a realistic dataset with missing values to simulate real-world messiness.
2.  **Preprocessing**: Build a strictly reproducible pipeline to handle missing data (Imputation) and scale features (Standardization).
3.  **Model Comparison**: Train `KMeans`, `MiniBatchKMeans`, and `GMM` across different cluster counts (K=3..6).
4.  **Evaluation**: Use **Silhouette Score**, **Calinski-Harabasz Index**, and **Inertia** to find the best K.
5.  **Stability Analysis**: Verify the chosen model's robustness by running it with multiple random seeds and checking the **Adjusted Rand Index (ARI)**.
6.  **Interpretation**: Translate mathematical clusters into business personas.

### 🎯 Expected Output
- A recommendation on the optimal number of clusters (K).
- A stability score confirming if the clusters are reliable.
- Business profiles for each segment (e.g. "Champions", "At-Risk").

### 🔹 Imports Explanation
#### 2.1 What the line does
Imports necessary libraries for data manipulation, visualization, machine learning, and metrics.

#### 2.2 Why it is used
- `numpy` & `pandas`: Foundation for data structures.
- `sklearn`: The industry standard library for traditional ML algorithms in Python.
- `matplotlib` & `seaborn`: For visualizing the metric trends.

#### 2.3 When to use it
At the start of every Data Science project.


#### 2.5 How to use it
Standard python import syntax.

#### 2.6 How it works internally
Loads the compiled C/C++ extensions (for numpy/sklearn) into memory for fast execution.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score, calinski_harabasz_score, adjusted_rand_score
import warnings

warnings.filterwarnings('ignore') # Clean up output for teaching

### 🔹 Data Generation Function

#### ⚙️ Function Arguments Explanation: `make_blobs`
- `n_samples`: Total data points (5000). More samples = more stable clusters.
- `n_features`: Number of columns (12). Represents metrics like "Logins", "PageViews", etc.
- `centers`: The true number of clusters to generate (5). In real life, we wouldn't know this.
- `cluster_std`: How spread out each blob is. Higher = harder to cluster.
- `random_state`: Seed for reproducibility.

#### 2.1 What the code does
Generates a synthetic matrix of data `X` and creates artificial missing values (NaNs) to mimic real-world dirty data.

#### 2.2 Why it is used
We don't have access to the company's internal database, so we simulate it to test our pipeline.


In [ ]:
def generate_saas_data(n_samples=5000, n_features=12, random_state=42):
    """
    Generates synthetic B2B SaaS data.
    Simulates features like Logins, FeatureAdoption, SupportInteractions, etc.
    """
    # Generate clean blobs
    X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=5, 
                      cluster_std=2.5, random_state=random_state)
    
    # Introduce some missing values (Dirty Data Simulation)
    rng = np.random.RandomState(random_state)
    mask = rng.rand(n_samples, n_features) < 0.05 # 5% probability of being missing
    X_missing = X.copy()
    X_missing[mask] = np.nan
    
    feature_names = [f'Feature_{i+1}' for i in range(n_features)]
    return X_missing, feature_names

### 🔹 Compare Models Function

This is the core logic engine.

#### ⚙️ Important Concepts
1. **Pipeline**: chaining `SimpleImputer` -> `StandardScaler`. We do this *outside* the loop here to speed up calculation, but typically it should be part of the prediction flow.
2. **Inertia**: Sum of squared distances to closest centroid. Lower is better (more compact).
3. **Silhouette Score**: How well separated clusters are. Ranges [-1, 1]. Higher is better.
4. **Calinski-Harabasz**: Ratio of dispersion. Higher is better.

In [ ]:
def compare_models(X, k_range, output_dir):
    results = []
    
    # Preprocess once for efficiency in this comparison loop
    preprocessor = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), # Fill NaNs with column mean
        ('scaler', StandardScaler())                 # Scale to mean=0, std=1
    ])
    X_processed = preprocessor.fit_transform(X)
    
    best_score = -1
    best_model_config = None

    models_to_test = ['KMeans', 'MiniBatchKMeans', 'GMM']
    
    print("Starting Model Comparison...")
    
    for k in k_range:
        for name in models_to_test:
            # Initialize Model
            if name == 'KMeans':
                model = KMeans(n_clusters=k, random_state=42, n_init=10)
            elif name == 'MiniBatchKMeans':
                model = MiniBatchKMeans(n_clusters=k, random_state=42, n_init=10)
            else: # GMM
                model = GaussianMixture(n_components=k, random_state=42)
            
            # Fit Model
            model.fit(X_processed)
            
            # Get Labels & Metrics
            if name == 'GMM':
                labels = model.predict(X_processed)
                inertia = np.nan # GMM minimizes log-likelihood, not inertia
            else:
                labels = model.labels_
                inertia = model.inertia_
            
            # Calculate Silhouette (Sampled for speed if N is huge)
            sil_score = silhouette_score(X_processed, labels, sample_size=1000, random_state=42)
            ch_score = calinski_harabasz_score(X_processed, labels)
            
            results.append({
                'Model': name,
                'K': k,
                'Inertia': inertia,
                'Silhouette': sil_score,
                'Calinski_Harabasz': ch_score
            })
            
            # Track Winner
            if sil_score > best_score:
                best_score = sil_score
                best_model_config = (name, k)
                
    return pd.DataFrame(results), best_model_config

### 🔹 Stability Analysis Function

#### 2.1 What the code does
Runs the *best* model configuration 5 times with DIFFERENT random seeds and compares the results.

#### 2.2 Why it is used
**Crucial Step**: In unsupervised learning, a "good" score might be a fluke of initialization. If we run it again tomorrow, will we get the same customer segments?
- If **ARI (Adjusted Rand Index)** is near 1.0: Stable. Safe to use.
- If **ARI** is near 0.0: Unstable. Random noise. Do NOT use.

#### 3.5 How to use `adjusted_rand_score`
`score = adjusted_rand_score(labels_1, labels_2)` compares two lists of cluster assignments.

In [ ]:
def stability_analysis(X, model_name, k):
    print(f"\nRunning Stability Analysis for {model_name} with K={k}...")
    
    # Preprocess again (simulating fresh runs)
    preprocessor = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    X_processed = preprocessor.fit_transform(X)
    
    seeds = [42, 1, 2, 3, 4]
    labels_list = []
    
    for seed in seeds:
        if model_name == 'KMeans':
            model = KMeans(n_clusters=k, random_state=seed, n_init=10)
        elif model_name == 'MiniBatchKMeans':
            model = MiniBatchKMeans(n_clusters=k, random_state=seed, n_init=10)
        else:
            model = GaussianMixture(n_components=k, random_state=seed)
        
        if model_name == 'GMM':
            labels = model.fit_predict(X_processed)
        else:
            model.fit(X_processed)
            labels = model.labels_
        labels_list.append(labels)
    
    # Pairwise comparison
    ari_scores = []
    for i in range(len(seeds)):
        for j in range(i + 1, len(seeds)):
            score = adjusted_rand_score(labels_list[i], labels_list[j])
            ari_scores.append(score)
            
    avg_stability = np.mean(ari_scores)
    return avg_stability

### 🔹 Main Execution

1. Generate Data.
2. Compare all models.
3. Pick the winner.
4. Check winner's stability.
5. Print Business Conclusion.

In [ ]:
# 1. Generate Data
X, feature_names = generate_saas_data()
print(f"Data Generated: {X.shape}")

# 2. Compare Models
k_range = [3, 4, 5, 6]
results_df, best_config = compare_models(X, k_range, output_dir=None)

# Display Top Results
print("\nModel Comparison Metrics:")
display(results_df.sort_values(by='Silhouette', ascending=False).head())

# 3. Best Model Info
best_model_name, best_k = best_config
print(f"\n🏆 Best Configuration: {best_model_name} with K={best_k}")

# 4. Check Stability
stability = stability_analysis(X, best_model_name, best_k)
print(f"\n⚓ Average Stability Score (ARI): {stability:.4f}")
if stability > 0.9:
    print("✅ Result: Extremely Robust Clusters.")
elif stability > 0.75:
    print("✅ Result: Stable Clusters.")
else:
    print("⚠️ Result: Unstable Clusters. Do not deploy.")